In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-and-test-data/imputated_test_data.csv
/kaggle/input/train-and-test-data/imputated_train_data.csv
/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import warnings

In [3]:
warnings.filterwarnings("ignore")


In [4]:
train = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
train_extra = pd.read_csv("/kaggle/input/playground-series-s5e2/training_extra.csv")
train_data = pd.concat([train, train_extra], axis=0, ignore_index=True)
test_data = pd.read_csv("/kaggle/input/playground-series-s5e2/test.csv")

Y_train = train_data[["Price"]]


In [5]:
len(Y_train)

3994318

In [ ]:
# Encode categorical features
categorical_features = ['Brand', 'Material', 'Size', 'Style', 'Color']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(train_data[categorical_features])
categorical_data = pd.DataFrame(categorical_encoded, index=train_data.index, columns=encoder.get_feature_names_out(categorical_features))


****Feature Extraction****

In [ ]:
# Encode categorical features for test data
categorical_encoded_test = encoder.fit_transform(test_data[categorical_features])
categorical_data_test = pd.DataFrame(categorical_encoded_test, columns=encoder.get_feature_names_out(categorical_features))

In [ ]:
numerical_features = ['id','Compartments', 'Weight Capacity (kg)']
binary_features = ['Laptop Compartment', 'Waterproof']


In [ ]:
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [ ]:
train_data[binary_features] = train_data[binary_features].replace({'Yes': 1, 'No': 0})
test_data[binary_features] = test_data[binary_features].replace({'Yes': 1, 'No': 0})


In [ ]:
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [ ]:
X_train = pd.concat([train_data[numerical_features + binary_features], categorical_data], axis=1)
X_test = pd.concat([test_data[numerical_features + binary_features], categorical_data_test], axis=1)

In [ ]:
display(X_train)

In [ ]:
X_train.drop('id', axis =1)

****Feature Scaling****

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)

print("Imputation complete using IterativeImputer!")


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use IterativeImputer (more intelligent but still efficient)
imputer = IterativeImputer(max_iter=10, random_state=42)

X_test = pd.DataFrame(imputer.fit_transform(X_test), columns=X_test.columns)

print("Imputation complete using IterativeImputer!")


In [ ]:
X_train.to_csv("/kaggle/working/imputated_train_data.csv", index=False)
X_test.to_csv("/kaggle/working/imputated_test_data.csv", index=False)


In [5]:
import pandas as pd
X_train = pd.read_csv('/kaggle/input/train-and-test-data/imputated_train_data.csv')
X_test = pd.read_csv('/kaggle/input/train-and-test-data/imputated_test_data.csv')

In [6]:
missing_count_per_column = X_test.isnull().sum()
print(missing_count_per_column)


id                      0
Compartments            0
Weight Capacity (kg)    0
Laptop Compartment      0
Waterproof              0
Brand_Adidas            0
Brand_Jansport          0
Brand_Nike              0
Brand_Puma              0
Brand_Under Armour      0
Brand_nan               0
Material_Canvas         0
Material_Leather        0
Material_Nylon          0
Material_Polyester      0
Material_nan            0
Size_Large              0
Size_Medium             0
Size_Small              0
Size_nan                0
Style_Backpack          0
Style_Messenger         0
Style_Tote              0
Style_nan               0
Color_Black             0
Color_Blue              0
Color_Gray              0
Color_Green             0
Color_Pink              0
Color_Red               0
Color_nan               0
dtype: int64


In [7]:
missing_count_per_column = X_train.isnull().sum()
print(missing_count_per_column)


id                      0
Compartments            0
Weight Capacity (kg)    0
Laptop Compartment      0
Waterproof              0
Brand_Adidas            0
Brand_Jansport          0
Brand_Nike              0
Brand_Puma              0
Brand_Under Armour      0
Brand_nan               0
Material_Canvas         0
Material_Leather        0
Material_Nylon          0
Material_Polyester      0
Material_nan            0
Size_Large              0
Size_Medium             0
Size_Small              0
Size_nan                0
Style_Backpack          0
Style_Messenger         0
Style_Tote              0
Style_nan               0
Color_Black             0
Color_Blue              0
Color_Gray              0
Color_Green             0
Color_Pink              0
Color_Red               0
Color_nan               0
dtype: int64


In [8]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [9]:
# Ensure your data is in NumPy array format
X_train = X_train.to_numpy() if hasattr(X_train, 'to_numpy') else X_train
Y_train = Y_train.to_numpy() if hasattr(Y_train, 'to_numpy') else Y_train
X_test = X_test.to_numpy() if hasattr(X_test, 'to_numpy') else X_test


**Random Forrest Initialization**

In [10]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
import tensorflow as tf

# Initialize the MirroredStrategy
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")


Number of devices: 1


In [12]:
!nvidia-smi


Tue Feb 18 02:49:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             30W /  250W |     259MiB /  16384MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


# Split into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Convert data to XGBoost DMatrix format
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

# Optimized XGBoost parameters
params =     { "objective": "reg:squarederror",
        "tree_method": "gpu_hist",  # Enable GPU
     'max_depth': None, 
      'learning_rate': 0.010521071617639474, 
      'subsample': 0.7382565786225133, 
      'colsample_bytree': 0.6610167338879633, 
      'reg_lambda': 4.6051558884581745, 
      'reg_alpha': 1.575531902929693, 
      'min_child_weight': 4, 
        'gamma': 0.009928420972646916}

# Train XGBoost on GPU
model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
Y_pred = model.predict(dval)

# Evaluate performance
rmse = np.sqrt(mean_squared_error(Y_val, Y_pred))
print(f"GPU R2 Score (XGBoost on Tesla P100): {rmse:.4f}")


GPU R2 Score (XGBoost on Tesla P100): 38.8991


In [17]:
dtest = xgb.DMatrix(X_test)


In [18]:
predictions = model.predict(dtest)
df = pd.DataFrame(predictions, columns=["Predictions"])

# Save DataFrame to an Excel file
file_path = "/kaggle/working/model_predictions.xlsx"
df.to_excel(file_path, index=False)

print(f"Excel file saved as {file_path}")

Excel file saved as /kaggle/working/model_predictions.xlsx


In [ ]:
display(test_data["id"])
file_ids = "/kaggle/working/model_ids.xlsx"
test_data["id"].to_excel(file_ids, index=False)


In [25]:
import xgboost as xgb
import numpy as np
import pandas as pd
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Convert data to XGBoost DMatrix format
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

# Define the objective function for Optuna
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "tree_method": "gpu_hist",  # Enable GPU
        "max_depth": trial.suggest_int("max_depth", 15, 25),  
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),  
        "subsample": trial.suggest_uniform("subsample", 0.6, 1.0),  
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.6, 1.0),  
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1, 100),  
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 0.1, 10),  
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),  
        "gamma": trial.suggest_loguniform("gamma", 1e-8, 1.0),  
        "random_state": 42
    }
    
    # Train model with current parameters
    model = xgb.train(params, dtrain, num_boost_round=1500, evals=[(dval, "eval")], 
                      early_stopping_rounds=40, verbose_eval=False)
    
    # Make predictions and evaluate RMSE
    Y_pred = model.predict(dval)
    rmse = np.sqrt(mean_squared_error(Y_val, Y_pred))
    
    return rmse

# Running Optuna for best hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# Print best parameters
best_params = study.best_params
print("\n🔹 Best XGBoost Parameters Found:", best_params)


[I 2025-02-17 08:46:23,460] A new study created in memory with name: no-name-2a325d61-1977-49cd-b915-97c48ac6a350
[I 2025-02-17 08:46:37,050] Trial 0 finished with value: 39.31129100983374 and parameters: {'max_depth': 20, 'learning_rate': 0.06387411525364202, 'subsample': 0.7056635066179081, 'colsample_bytree': 0.980565800794401, 'reg_lambda': 2.027883789488466, 'reg_alpha': 0.36178979492184454, 'min_child_weight': 5, 'gamma': 0.00018420669327376784}. Best is trial 0 with value: 39.31129100983374.
[I 2025-02-17 08:47:00,663] Trial 1 finished with value: 39.35164431888976 and parameters: {'max_depth': 24, 'learning_rate': 0.05853426829861063, 'subsample': 0.9988833865959503, 'colsample_bytree': 0.714567365953453, 'reg_lambda': 1.9818779698354536, 'reg_alpha': 2.207171468197212, 'min_child_weight': 7, 'gamma': 1.4022781913821426e-08}. Best is trial 0 with value: 39.31129100983374.
[I 2025-02-17 08:47:16,202] Trial 2 finished with value: 40.463752366397095 and parameters: {'max_depth': 2


🔹 Best XGBoost Parameters Found: {'max_depth': 17, 'learning_rate': 0.011240470577189023, 'subsample': 0.6767216803352312, 'colsample_bytree': 0.8871906286538357, 'reg_lambda': 87.64840683931247, 'reg_alpha': 0.4760401422425682, 'min_child_weight': 10, 'gamma': 6.092732375329185e-05}
